In [ ]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub langchain langgraph tavily-python llama-index llama-parse llama-index-llms-groq llama-index-embeddings-fastembed fastembed  llama-index-vector-stores-qdrant

In [ ]:
# pdf data
!wget "https://lawcommission.gov.np/en/wp-content/uploads/2021/01/Constitution-of-Nepal.pdf"

--2024-06-14 03:06:36--  https://lawcommission.gov.np/en/wp-content/uploads/2021/01/Constitution-of-Nepal.pdf
Resolving lawcommission.gov.np (lawcommission.gov.np)... 202.45.146.189
Connecting to lawcommission.gov.np (lawcommission.gov.np)|202.45.146.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3516131 (3.4M) [application/pdf]
Saving to: ‘Constitution-of-Nepal.pdf’

Constitution-of-Nep 100%[===================>]   3.35M  1.39MB/s    in 2.4s    

2024-06-14 03:06:40 (1.39 MB/s) - ‘Constitution-of-Nepal.pdf’ saved [3516131/3516131]



In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()
os.environ['LLAMA_PARSE_API_KEY']="llx-v8MmVwI9pifjwD3c0AI4KeaK6BQtIPz1RrZrIVisC2mK7Mx8"

from llama_parse import LlamaParse
llama_parse_documents = LlamaParse(api_key="llx-v8MmVwI9pifjwD3c0AI4KeaK6BQtIPz1RrZrIVisC2mK7Mx8", result_type="markdown").load_data("/content/Constitution-of-Nepal.pdf")

Started parsing the file under job_id cac11eca-1d95-4786-9a75-13d817310eb4


In [ ]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [ ]:
# model
from llama_index.llms.groq import Groq

os.environ["GROQ_API_KEY"]='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'
llm1 = Groq(model="mixtral-8x7b-32768", api_key="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")

In [ ]:
from llama_index.core import Settings
Settings.llm=llm1
Settings.embed_model=embed_model

In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
import qdrant_client

client = qdrant_client.QdrantClient(
    "https://157df9e2-c80f-418f-bfb5-d75133f04154.us-east4-0.gcp.cloud.qdrant.io",
    api_key="8J1gEvOSEqii300JBP5pugw5E35kEbELwsleEJbMjuXS9rfICC6E5A",
)

vector_store = QdrantVectorStore(client=client, collection_name="documents")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index =VectorStoreIndex.from_documents(documents=llama_parse_documents,storage_context=storage_context,show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/85 [00:00<?, ?it/s]

In [111]:
from IPython.display import display, Markdown

In [123]:
retriever = index.as_retriever(search_kwargs={"k":3})
# docs = retriever.retrieve(question)
# query_engine = index.as_query_engine()
# response = query_engine.query("what specific rights can nrn's exercise?")
# display(Markdown(response.response))

In [ ]:
os.environ['LANGCHAIN_TRACING-V2']='true'
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']='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'

In [ ]:
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    model="mixtral-8x7b-32768",
    api_key="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"
)

In [ ]:
# router
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""system You are an expert at routing a
    user question to a vectorstore or web_search. Use the vectorstore if you think the provided document has the answer to the question.
    It does not need to be a stringent test. Otherwise, use web_search. The goal is to find whether the document is relevant to the question or not.
    Return a JSON with a single key 'datasource' and no preamble or explanation.

    user
    here is the question and document.
    question = {question}
    document = {document}""",
    input_variables=["question", "document"],
)
start = time.time()
question_router = prompt | llm | JsonOutputParser()
question = "a person not the member of federal parliament, can he be still appointed as a minister?"
docs = retriever.retrieve(question)
docs = [d.text for d in docs]
# print(docs)
response = question_router.invoke({"question": question,"document":docs})
print(response)
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

{'datasource': 'vectorstore'}
The time required to generate response by Router Chain in seconds:0.8447427749633789


In [ ]:
# generation
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
start = time.time()
rag_chain = prompt | llm | StrOutputParser()
question = "a person not the member of federal parliament, can he be still appointed as a minister?"
docs = retriever.retrieve(question)
docs = [d.text for d in docs]
generation = rag_chain.invoke({"context":docs,"question": question})
print(generation)
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

Yes, a person who is not a member of the Federal Parliament can be appointed as a Minister, but they must become a member of the Federal Parliament within six months from the date of taking the oath. If they fail to do so, they cannot be reappointed to the office of Minister during the term of the then House of Representatives.
The time required to generate response by Router Chain in seconds:0.6023774147033691


In [ ]:
# retrieval grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
    of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)
start = time.time()
retrieval_grader = prompt | llm | JsonOutputParser()
question = "how a government forms?"
docs = retriever.retrieve(question)
docs = [doc.text for doc in docs]
print(retrieval_grader.invoke({"question": question, "document": docs}))
end = time.time()
print(f"The time required to generate response by the retrieval grader in seconds:{end - start}")

{'score': 'yes'}
The time required to generate response by the retrieval grader in seconds:0.8651895523071289


In [ ]:
# hallucination grader
prompt = PromptTemplate(
    template="""system You are a grader assessing whether
    an answer is grounded in or supported by a set of facts. Give a score 'yes' or 'no' to indicate whether the generation is based on the provided document or not. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.

    user
    Here are the facts:
    -------
    {documents}
    -------
    Here is the answer:
    {generation}""",
    input_variables=["generation", "documents"],
)
start = time.time()
hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader_response = hallucination_grader.invoke({"documents": docs, "generation": generation})
end = time.time()
print(f"The time required to generate response by the generation chain in seconds:{end - start}")
print(hallucination_grader_response)


The time required to generate response by the generation chain in seconds:0.5826699733734131
{'score': 'no'}


In [ ]:
# answer grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is useful to resolve a question. Give a  score 'yes' or 'no' to indicate whether the answer is
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = prompt | llm | JsonOutputParser()
answer_grader_response = answer_grader.invoke({"question": question,"generation": generation})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

The time required to generate response by the answer grader in seconds:0.39989352226257324
{'score': 'yes'}


In [ ]:
# web search

from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = "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"
web_search_tool = TavilySearchResults(k=3)

In [ ]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]

In [125]:
from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.retrieve(question)
    documents = [doc.text for doc in documents]
    return {"documents": documents, "question": question}
#
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}
#
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}
#
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    print(docs)
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

In [126]:
# conditional edges
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    documents = retriever.retrieve(question)
    print(question)
    source = question_router.invoke({"question": question,"document":documents})
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

In [127]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


In [128]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [129]:
# adding notes
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

In [130]:
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

In [131]:
app = workflow.compile()

In [132]:
from pprint import pprint
inputs = {"question":"what specific rights can nrn's exercise?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
what specific rights can nrn's exercise?
vectorstore
---ROUTE QUESTION TO RAG---
---RETRIEVE---
'Finished running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
'Finished running: grade_documents:'
---WEB SEARCH---
[{'url': 'https://meroadalat.com/rights-of-nrn-in-nepal/', 'content': 'Benefits of having an NRN Card. Travel Freedom: NRNs from non-SAARC countries can enter and stay in Nepal without a visa. Money Benefits: Open special bank accounts, invest, and enjoy tax breaks on certain money transfers. Land and Home: Buy and sell some types of land and keep family properties in Nepal.'}, {'url': 'https://elitelegal.com.np/resources/economic-social-and-cultural-rights-of-non-resident-nepalese/', 'content': 'One significant aspect of the rights of NRNs in Nepal is their ability to own property. A person holding a Non